In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from backtest import backtest
from plotting import plot_fills, make_table
from downloader import load_hlc_cache
from pure_funcs import (
    denumpyize,
    numpyize,
    candidate_to_live_config,
    calc_spans,
    analyze_fills,
    round_dynamic,
    round_values,
    sort_dict_keys,
)
from procedures import (
    dump_live_config,
    load_live_config,
    add_argparse_args,
    prepare_backtest_config,
    make_get_filepath,
)
from time import time
import sys
import argparse
import pprint
import matplotlib.pyplot as plt
import json
import pandas as pd
import numpy as np

In [ ]:
plt.rcParams["figure.figsize"] = [24, 13.5]
plt.rcParams["figure.facecolor"] = "w"
pd.set_option("display.precision", 10)

In [ ]:
class Args:
    def __init__(self):
        self.backtest_config_path = "configs/backtest/default.hjson"
        self.exchange = "binance"
        self.symbol = "XMRUSDT"
        self.market_type = "futures"
        self.user = "binance_01"
        self.start_date = "2021-05-01"
        self.end_date = "2023-03-10"
        self.starting_balance = 10000.0
        self.starting_configs = ""
        self.base_dir = "backtests"


config = await prepare_backtest_config(Args())
config["adg_n_subdivisions"] = 20  # see configs/optimize/default.hjson for details

sts = time()
data = load_hlc_cache(
    config["symbol"],
    config["start_date"],
    config["end_date"],
    base_dir=config["base_dir"],
    spot=config["spot"],
    exchange=config["exchange"],
)
prices = data[:, 3]
highs = data[:, 1]
lows = data[:, 2]
closes = data[:, 3]
timestamps = data[:, 0]
config["n_days"] = (timestamps[-1] - timestamps[0]) / (1000 * 60 * 60 * 24)

print(f"millis to load {len(prices)} ticks {(time() - sts) * 1000:.0f}ms")

In [ ]:
df = pd.DataFrame({"timestamp": timestamps, "price": prices}).set_index("timestamp")
df.price.iloc[::100].plot(title="Coin Price", xlabel="Time", ylabel="Price")

In [ ]:
hand_tuned = {
    "config_name": "emas_0days",
    "long": {
        "delay_between_fills_minutes_close": 703.3,
        "delay_between_fills_minutes_entry": 187.5,
        "delay_weight_close": 3.989,
        "delay_weight_entry": 8.921,
        "ema_dist_entry": 0.0008584,
        "ema_dist_close": 0.007752,
        "ema_span_0": 954.0,
        "ema_span_1": 124.3,
        "enabled": True,
        "markup_range": 0.06996,
        "min_markup": 0.02848,
        "n_close_orders": 14,
        "qty_pct_close": 0.09392,
        "qty_pct_entry": 0.01674,
        "wallet_exposure_limit": 1.0,
        "we_multiplier_close": 7.798,
        "we_multiplier_entry": 6.95,
    },
    "short": {
        "delay_between_fills_minutes_close": 1172.0,
        "delay_between_fills_minutes_entry": 1351.0,
        "delay_weight_close": 2.832,
        "delay_weight_entry": 15.12,
        "ema_dist_entry": 0.004033,
        "ema_dist_close": 0.005519,
        "ema_span_0": 1251.0,
        "ema_span_1": 1830.0,
        "enabled": True,
        "markup_range": 0.06225,
        "min_markup": 0.06723,
        "n_close_orders": 5,
        "qty_pct_close": 0.06165,
        "qty_pct_entry": 0.01561,
        "wallet_exposure_limit": 1.0,
        "we_multiplier_close": 9.278,
        "we_multiplier_entry": 49.73,
    },
}

config_to_test = {**config, **hand_tuned}

In [ ]:
sts = time()
fills_long, fills_short, stats = backtest(config_to_test, data)
elapsed = time() - sts
print(f"seconds elapsed {elapsed:.4f}")
longs, shorts, sdf, analysis = analyze_fills(fills_long, fills_short, stats, config_to_test)
table = make_table({**config_to_test, **analysis})
print(table)

In [ ]:
sdf

In [ ]:
sdf.balance_long.plot()
sdf.equity_long.plot(title="Balance and equity long", xlabel="Time", ylabel="Balance")

In [ ]:
sdf.balance_short.plot()
sdf.equity_short.plot(title="Balance and equity short", xlabel="Time", ylabel="Balance")

In [ ]:
if len(longs) > 0:
    plot_fills(df, longs, plot_whole_df=True, title="Overview Fills Long")

In [ ]:
if len(shorts) > 0:
    plot_fills(df, shorts, plot_whole_df=True, title="Overview Fills Short")

In [ ]:
wes = pd.DataFrame(
    {"WE_short": (sdf.wallet_exposure_short.abs() * -1), "WE_long": sdf.wallet_exposure_long}
)
wes.plot(title="Wallet exposures", xlabel="Time", ylabel="Wallet Exposure")

In [ ]:
sdf.price.plot(title="Average entry price", xlabel="Time", ylabel="Price")
sdf[sdf.psize_long != 0].pprice_long.replace(0.0, np.nan).plot()
sdf[sdf.psize_short != 0].pprice_short.replace(0.0, np.nan).plot()

In [ ]:
lpprices = sdf[sdf.psize_long != 0.0]
PADistance_long = (lpprices.pprice_long - lpprices.price).abs() / lpprices.price
print(f"Price action distance long mean {PADistance_long.mean():.6f} std {PADistance_long.std():.6f}")
PADistance_long.plot(title="Price action distance", xlabel="Time", ylabel="Price action distance")

In [ ]:
spprices = sdf[sdf.psize_short != 0.0]
PADistance_short = (spprices.pprice_short - spprices.price).abs() / spprices.price
print(f"Mean price action distance short {PADistance_short.mean():.6f}")
PADistance_short.plot(title="Price action distance", xlabel="Time", ylabel="Price action distance")

In [ ]:
spans_long = sorted(
    [
        config_to_test["long"]["ema_span_0"],
        (config_to_test["long"]["ema_span_0"] * config_to_test["long"]["ema_span_1"]) ** 0.5,
        config_to_test["long"]["ema_span_1"],
    ]
)
emas_long = pd.DataFrame(
    {f"ema_{span}": df.price.ewm(span=span, adjust=False).mean() for span in spans_long},
    index=df.index,
)
ema_bands_long = pd.DataFrame(
    {
        "ema_band_lower": emas_long.min(axis=1) * (1 - config_to_test["long"]["ema_dist_entry"]),
        "ema_band_upper": emas_long.max(axis=1) * (1 + config_to_test["long"]["ema_dist_close"]),
    },
    index=df.index,
)
df_emas_long = df.join(ema_bands_long)
df_emas_long.price.plot(style="y-")
df_emas_long.ema_band_lower.plot(style="b--")
df_emas_long.ema_band_upper.plot(style="r--")

In [ ]:
spans_short = sorted(
    [
        config_to_test["short"]["ema_span_0"],
        (config_to_test["short"]["ema_span_0"] * config_to_test["short"]["ema_span_1"]) ** 0.5,
        config_to_test["short"]["ema_span_1"],
    ]
)
emas_short = pd.DataFrame(
    {f"ema_{span}": df.price.ewm(span=span, adjust=False).mean() for span in spans_short},
    index=df.index,
)
ema_bands_short = pd.DataFrame(
    {
        "ema_band_lower": emas_short.min(axis=1) * (1 - config_to_test["short"]["ema_dist_close"]),
        "ema_band_upper": emas_short.max(axis=1) * (1 + config_to_test["short"]["ema_dist_entry"]),
    },
    index=df.index,
)
df_emas_short = df.join(ema_bands_short)
df_emas_short.price.plot(style="y-")
df_emas_short.ema_band_lower.plot(style="b--")
df_emas_short.ema_band_upper.plot(style="r--")